In [45]:
import requests
import passwords as pw
import pandas as pd
import time
from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
def get_puuid(game_name, tag_line, api_key):
    api_url = (f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key=" + api_key)
    resp = requests.get(api_url)
    player_info = resp.json()
    return player_info['puuid']


In [57]:
get_puuid('Pobelter', 'NA1', pw.API_KEY)

'3JL0_mjV_ql6I4cONFQ2wu4Lxb2pA1qzXlQPGR-zQYxbAnZ-np57GDRVvCFCGKiO3vnfPlQNkISi6g'

In [3]:
def get_active_game(puuid, api_key):
    headers = {"X-Riot-Token": api_key}
    api_url = (f"https://na1.api.riotgames.com/lol/spectator/v5/active-games/by-summoner/{puuid}")
    resp = requests.get(api_url, headers = headers)
    game_data = resp.json()

    champion_ids = [player['championId'] for player in game_data['participants']]
    return champion_ids


In [4]:
def champion_names(champ_ids):
    version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
    api_url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json"
    champs = requests.get(api_url).json()["data"]
    id_to_name = {int(name["key"]): champ for champ, name in champs.items()}
    
    champ_names = [id_to_name.get(id, "Unknown") for id in champ_ids]
    print("Champions in game:", champ_names)

In [32]:
def get_match_ids(start, count, puuid, api_key):
    api_url = (f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}&api_key={api_key}")
    resp = requests.get(api_url)
    match_ids = resp.json()
    return match_ids

In [33]:
def get_match_data(match_id, api_key):
    api_url = ("https://americas.api.riotgames.com/lol/match/v5/matches/" + match_id +
               "?api_key=" + api_key)
    try:
        resp = requests.get(api_url)
        match_data = resp.json()
        return match_data
    except:
        return None

In [ ]:
def pull_matches(start, count = 100):
    dataframe = []
    match_ids = get_match_ids(start, count, pw.pob_PUUID, pw.API_KEY)
    print(match_ids)
    for match in match_ids:
        match_data = get_match_data(match, pw.API_KEY)
        if not match_data or 'info' not in match_data:
            continue

        participants = match_data['info']['participants']
        
        player = next((p for p in participants if p['puuid'] == pw.pob_PUUID), None)
        player_team = player['teamId']
        
        ally_champs = [p['championName'] for p in participants if p['teamId'] == player_team]
        enemy_champs = [p['championName'] for p in participants if p['teamId'] != player_team]
        
        dataframe.append({
            'match_id': match,
            'ally_champs': ally_champs,
            'enemy_champs': enemy_champs
        })
    
    return pd.DataFrame(dataframe)






In [81]:
final_df = pd.DataFrame(columns = ['match_id', 'ally_champs', 'enemy_champs'])

In [82]:
iterations = 10
start = 0
for i in range(iterations):
    temp = pull_matches(start, 100)
    start += 100
    final_df = pd.concat([temp, final_df], ignore_index = True)
    time.sleep(125)


['NA1_5311207389', 'NA1_5311182278', 'NA1_5311153226', 'NA1_5311129503', 'NA1_5311108877', 'NA1_5311074538', 'NA1_5311057970', 'NA1_5311038011', 'NA1_5311020103', 'NA1_5307377806', 'NA1_5307348286', 'NA1_5307326746', 'NA1_5307305138', 'NA1_5307278676', 'NA1_5307264991', 'NA1_5297341663', 'NA1_5297309040', 'NA1_5297275657', 'NA1_5297254056', 'NA1_5297229937', 'NA1_5297193392', 'NA1_5297164219', 'NA1_5296226791', 'NA1_5296191049', 'NA1_5296147274', 'NA1_5296118301', 'NA1_5296084090', 'NA1_5296061364', 'NA1_5296029703', 'NA1_5296004233', 'NA1_5295983342', 'NA1_5295963246', 'NA1_5295934236', 'NA1_5295917896', 'NA1_5295903312', 'NA1_5295880545', 'NA1_5294108653', 'NA1_5294065372', 'NA1_5294023475', 'NA1_5293985548', 'NA1_5292699678', 'NA1_5292668414', 'NA1_5292636730', 'NA1_5292588079', 'NA1_5286559276', 'NA1_5286527529', 'NA1_5286496319', 'NA1_5286473981', 'NA1_5286432583', 'NA1_5286404934', 'NA1_5286366777', 'NA1_5286336812', 'NA1_5285974436', 'NA1_5285945316', 'NA1_5285913989', 'NA1_5285

In [84]:
final_df

,match_id,ally_champs,enemy_champs
0,NA1_5163600177,"[Graves, Hwei, Kennen, Aphelios, TahmKench]","[Poppy, Naafiri, Illaoi, Xayah, Ekko]"
1,NA1_5160296390,"[Ambessa, Ivern, Akali, KogMaw, Blitzcrank]","[Lulu, Kindred, Yasuo, Jhin, Nautilus]"
2,NA1_5160265488,"[Renekton, Ivern, Jayce, Syndra, Leona]","[Gragas, Diana, Yasuo, Xayah, Rakan]"
3,NA1_5160220589,"[Urgot, Ivern, Ambessa, Jhin, Veigar]","[Brand, Kayn, Naafiri, Yasuo, Rell]"
4,NA1_5160158943,"[Aatrox, Khazix, Jayce, Hwei, Bard]","[Udyr, Nidalee, Yasuo, Smolder, Nami]"
...,...,...,...
900,NA1_5278817897,"[MasterYi, Ashe, Pantheon, Yorick, Tristana, A...","[Renata, Sivir, Kaisa, Nunu, MasterYi, Illaoi,..."
901,NA1_5278806250,"[Evelynn, Briar, Lulu, Samira, Draven, Hecarim...","[Belveth, AurelionSol, Gnar, Lux, Warwick, Jhi..."
902,NA1_5278796589,"[MasterYi, Nami, Heimerdinger, Xayah, Kaisa, T...","[Yasuo, Ornn, Smolder, Shen, Singed, Sett, Ekk..."
903,NA1_5278695422,"[Ambessa, Viego, Sylas, AurelionSol, Leona]","[Shen, Naafiri, Neeko, Tristana, Janna]"


In [18]:
def get_all_champs():
    version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
    api_url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json"

    champs_url = f'https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json'
    resp = requests.get(champs_url).json()

    champ_data = resp['data']
    champ_names = list(champ_data.keys())

    return champ_names

In [86]:
final_df = final_df.drop_duplicates(subset = ['match_id'])
final_df

,match_id,ally_champs,enemy_champs
0,NA1_5163600177,"[Graves, Hwei, Kennen, Aphelios, TahmKench]","[Poppy, Naafiri, Illaoi, Xayah, Ekko]"
1,NA1_5160296390,"[Ambessa, Ivern, Akali, KogMaw, Blitzcrank]","[Lulu, Kindred, Yasuo, Jhin, Nautilus]"
2,NA1_5160265488,"[Renekton, Ivern, Jayce, Syndra, Leona]","[Gragas, Diana, Yasuo, Xayah, Rakan]"
3,NA1_5160220589,"[Urgot, Ivern, Ambessa, Jhin, Veigar]","[Brand, Kayn, Naafiri, Yasuo, Rell]"
4,NA1_5160158943,"[Aatrox, Khazix, Jayce, Hwei, Bard]","[Udyr, Nidalee, Yasuo, Smolder, Nami]"
...,...,...,...
900,NA1_5278817897,"[MasterYi, Ashe, Pantheon, Yorick, Tristana, A...","[Renata, Sivir, Kaisa, Nunu, MasterYi, Illaoi,..."
901,NA1_5278806250,"[Evelynn, Briar, Lulu, Samira, Draven, Hecarim...","[Belveth, AurelionSol, Gnar, Lux, Warwick, Jhi..."
902,NA1_5278796589,"[MasterYi, Nami, Heimerdinger, Xayah, Kaisa, T...","[Yasuo, Ornn, Smolder, Shen, Singed, Sett, Ekk..."
903,NA1_5278695422,"[Ambessa, Viego, Sylas, AurelionSol, Leona]","[Shen, Naafiri, Neeko, Tristana, Janna]"


In [87]:
final_df.to_csv('matches.csv', index = False)

In [88]:
#One Hot Encoding

all_champs = get_all_champs()

mlb_allies = MultiLabelBinarizer(classes=all_champs)
mlb_enemies = MultiLabelBinarizer(classes=all_champs)

ally_encoded = pd.DataFrame(mlb_allies.fit_transform(final_df['ally_champs']),
                            columns=[f'ally_{c}' for c in mlb_allies.classes_],
                            index=final_df.index)

enemy_encoded = pd.DataFrame(mlb_enemies.fit_transform(final_df['enemy_champs']),
                             columns=[f'enemy_{c}' for c in mlb_enemies.classes_],
                             index=final_df.index)

one_hot = pd.concat([final_df[['match_id']], ally_encoded, enemy_encoded], axis=1)

one_hot


c:\Users\minaz\OneDrive\Desktop\personal\win_predictor\env\Lib\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['FiddleSticks'] will be ignored
  warnings.warn(


,match_id,ally_Aatrox,ally_Ahri,ally_Akali,ally_Akshan,ally_Alistar,ally_Ambessa,ally_Amumu,ally_Anivia,ally_Annie,...,enemy_Yone,enemy_Yorick,enemy_Yuumi,enemy_Zac,enemy_Zed,enemy_Zeri,enemy_Ziggs,enemy_Zilean,enemy_Zoe,enemy_Zyra
0,NA1_5163600177,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NA1_5160296390,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NA1_5160265488,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NA1_5160220589,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NA1_5160158943,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,NA1_5278817897,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
901,NA1_5278806250,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
902,NA1_5278796589,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
903,NA1_5278695422,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
one_hot.to_csv('one_hot', index = False)